In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import regex as re

from prophet import Prophet
from prophet.plot import add_changepoints_to_plot
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
from prophet.plot import plot_cross_validation_metric


from statsmodels.tsa.seasonal import seasonal_decompose
#from learntools.time_series.utils import plot_periodogram, seasonal_plot

import sktime
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.utils.plotting import plot_series
from sktime.performance_metrics.forecasting import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error

Functions

In [0]:
#################
### Functions ###
#################

def get_color(name, number):
    pal = list(sns.color_palette(palette=name, n_colors=number).as_hex())
    return pal

def add_datepart(df, fldname, drop=True, time=False):
    "Helper function that adds columns relevant to a date."
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop: df.drop(fldname, axis=1, inplace=True)
    
def holiday(name):
    dset = holidays_events[holidays_events.locale == name]
    dset = dset[['date','var']].reset_index(drop=True)
    dset.rename(columns = {'var':name}, inplace = True)
    dset = dset.drop_duplicates()
    return dset    

########################
### Cross validation ###
########################

def cv_exam(model, test_df):
    
    forecast = model.predict(test_df)
    df_cv = cross_validation(model, initial='730 days', period='15 days', horizon = '30 days')
    df_p = performance_metrics(df_cv)
    df_cv['residual'] = df_cv['y'] - df_cv['yhat']
    prop_test = pd.merge(test_df,forecast[['yhat','ds']],on=['ds'],how='inner')
    prop_test.index = y_test.index
    
    return df_cv, df_p, forecast, prop_test

def perf_vals(dset, meas = 'smape'):
    rmse_add = dset.groupby(['Model'])[meas].mean()
    rmse_add = pd.DataFrame(rmse_add)
    a = rmse_add.sort_values(by = [meas])
    
    rmse_add = rmse_add.reset_index()
    model = rmse_add[rmse_add[meas] == min(rmse_add[meas])]
    print(f"The best model is the {model['Model'].values[0]} model")

    return a

In [0]:
spark.conf.set("fs.azure.account.key.ifsandboxstorage.dfs.core.windows.net", dbutils.secrets.get(scope="if-databricks-scope", key="if-storage-key"))

In [0]:
#import pickle with open('FILEPATH/store_sales_by_product_simple.pickle', 'rb') as f:
#    data = pickle.load(f) 

In [0]:
file_location = "experimental dataset/store-sales/stores.csv"
csvFile = "abfss://raw@ifsandboxstorage.dfs.core.windows.net/" + file_location
df_raw = spark.read.option("header", "true").options(inferSchema='True',delimiter=',').csv(csvFile)
stores = df_raw.toPandas()
stores.head()

In [0]:
file_location = "experimental dataset/store-sales/train.csv"
csvFile = "abfss://raw@ifsandboxstorage.dfs.core.windows.net/" + file_location
df_raw = spark.read.option("header", "true").options(inferSchema='True',delimiter=',').csv(csvFile)
train = df_raw.toPandas()
train.head()

In [0]:
file_location = "experimental dataset/store-sales/store_sales_by_product_simple.pickle"
csvFile = "abfss://raw@ifsandboxstorage.dfs.core.windows.net/" + file_location
df_raw = pd.read_pickle(re.sub(':', '', f'/{csvFile}')
#train = df_raw.toPandas()
df_raw.head()

In [0]:
#######################
### Sales by family ###
#######################

item_count_m = train.groupby(['family'])['sales'].sum()
item_count_m = item_count_m.reset_index()
item_count_m = pd.DataFrame(item_count_m)
item_count_m = item_count_m.sort_values(['sales'],ascending = False)
item_count_ma = item_count_m.iloc[:20,:]
item_count_ma = item_count_ma.reset_index(drop=True)
item_count_ma.head()

In [0]:
pal_vi = get_color('viridis_r', len(item_count_ma))
pal_plas = get_color('plasma_r', len(item_count_ma))
pal_spec = get_color('Spectral', len(item_count_ma))
pal_hsv = get_color('hsv', len(item_count_ma))

plt.gcf().set_size_inches(12, 12)
sns.set_style('darkgrid')

#set max value
max_val = max(item_count_ma['sales'])*1.01
ax = plt.subplot(projection='polar')

for i in range(len(item_count_ma)):
    ax.barh(i, list(item_count_ma['sales'])[i]*2*np.pi/max_val,
            label=list(item_count_ma['family'])[i], color=pal_plas[i])

#set the subplot 
ax.set_theta_zero_location('N')
ax.set_theta_direction(1)
ax.set_rlabel_position(0)
ax.set_thetagrids([], labels=[])
ax.set_rgrids(range(len(item_count_ma)), labels= item_count_ma['family'])

#set the projection
ax = plt.subplot(projection='polar')
plt.legend(bbox_to_anchor=(1, 1), loc=2)
plt.show()

In [0]:
print(train.shape)
train = train[train.family.isin(list(set(item_count_ma.family))[0:5]) & (train.store_nbr == 50)]
print(train.shape)

In [0]:
file_location = "experimental dataset/store-sales/oil.csv"
csvFile = "abfss://raw@ifsandboxstorage.dfs.core.windows.net/" + file_location
df_raw = spark.read.option("header", "true").options(inferSchema='True',delimiter=',').csv(csvFile)
oil = df_raw.toPandas()
oil.head()

In [0]:
file_location = "experimental dataset/store-sales/holidays_events.csv"
csvFile = "abfss://raw@ifsandboxstorage.dfs.core.windows.net/" + file_location
df_raw = spark.read.option("header", "true").options(inferSchema='True',delimiter=',').csv(csvFile)
holidays_events = df_raw.toPandas()
holidays_events.head()

In [0]:
##################
### Train data ###
##################

train = pd.merge(train,stores,on = 'store_nbr',how = 'left')
add_datepart(train, 'date', drop = False)
train['month_date'] = pd.to_datetime(train[['Year', 'Month']].assign(DAY=1))
print(train.shape)
# (3000888, 31)
train.head()

In [0]:
### Oil ###

miss_df = pd.DataFrame(oil.isna().mean() * 100)
#miss_df = miss_df[miss_df.iloc[:,0] != 0] 
print(miss_df)
# 3.530378
oil = oil[~oil.dcoilwtico.isna()]
oil.date = pd.to_datetime(oil.date).dt.date
oil.date = oil.date.astype('datetime64[ns]')
print(oil.shape)
oil.head()

In [0]:
print(holidays_events.shape)
holidays_events.date = pd.to_datetime(holidays_events.date).dt.date
holidays_events.date = holidays_events.date.astype('datetime64[ns]')
holidays_events_nd = holidays_events[['date']].drop_duplicates()
print(holidays_events_nd.shape)
holidays_events.head()

In [0]:
for name in list(holidays_events.columns):    
    print(f" There are {holidays_events[name].nunique()} unique {name} ids")
    if holidays_events[name].nunique() < 25:
        print(list(set(holidays_events[name])))

In [0]:
for name in list(set(holidays_events['type'])):
    print(name,holidays_events[holidays_events.type == str(name)].shape[0])

In [0]:
hol_list = holidays_events[holidays_events.type == 'Holiday']
hol_list = list(set(hol_list.date))
holi_list = [d.strftime('%d-%m-%Y') for d in hol_list]

In [0]:
print(holidays_events.shape)
holidays_events = holidays_events[(holidays_events.type != 'Work Day')  & (holidays_events.transferred != True)]
holidays_events['var'] = 1
print(holidays_events.shape)

national = holiday('National')
regional = holiday('Regional')
local = holiday('Local')
print(national.shape,regional.shape,local.shape)


In [0]:
####################
### Remerge data ###
####################

train.date = pd.to_datetime(train.date).dt.date
train.date = train.date.astype('datetime64[ns]')
print(train.shape)
train = pd.merge(train,oil,on = 'date',how = 'left')
train["dcoilwtico"] = train["dcoilwtico"].fillna(train.groupby('month_date')['dcoilwtico'].transform('mean'))
print(train.shape)

train = pd.merge(train,national,on = 'date',how = 'left')
print(train.shape)
train = pd.merge(train,regional,on = 'date',how = 'left')
print(train.shape)
train = pd.merge(train,local,on = 'date',how = 'left')
print(train.shape)
for name in ['National','Regional','Local']:
    train[name] = train[name].fillna(0)
print(train.shape)

In [0]:
train.head()

Analysis

In [0]:
############################
### Time series analysis ###
############################

#groc_data1 = train[(train.family == 'GROCERY I') & (train.store_nbr == 50)]
groc_data1 = train.copy()

print(groc_data1.shape)
print(list(groc_data1.columns))
groc_data1.head()

In [0]:
print(list(set(groc_data1.family)))

In [0]:
#########################
### Percentage change ###
#########################

plt.rc('font',size=10)
grid = gridspec.GridSpec(3,2)
plt.figure(figsize=(19,15))
plt.subplots_adjust(wspace=0.4,hspace=0.3)
      
for idx, name in enumerate(list(set(groc_data1.family))):
    ax = plt.subplot(grid[idx])
    dset = groc_data1[groc_data1.family == name]
    dset['Change'] = dset.sales.div(groc_data1.sales.shift())

    sns.lineplot(x="Dayofyear", y="Change", hue='Year', markers=True, data=dset)
    ax.set(xlabel='Dates', ylabel='Sales count percentage change')

Seasonality check

In [0]:
#seasonal_plot(groc_data1, y='sales', period='Year', freq='Month')

In [0]:
#seasonal_plot(groc_data1, y='sales', period='Year', freq='Week')

In [0]:
#seasonal_plot(groc_data1, y='sales', period='Week', freq='Dayofweek')

Decomposition

In [0]:
#####################
### Decomposition ###
#####################

plt.rc('font',size=10)
grid = gridspec.GridSpec(3,2)
plt.figure(figsize=(19,15))
plt.subplots_adjust(wspace=0.4,hspace=0.3)
      
for idx, name in enumerate(list(set(groc_data1.family))):
    ax = plt.subplot(grid[idx])
    dset = groc_data1[groc_data1.family == name]
    dset = dset[['date','sales']]
    dset = dset.reset_index(drop=True)
    dset = dset.sort_values(['date','sales'])
    dset.set_index('date',inplace=True)
    dset.index = pd.to_datetime(dset.index)
    ax.set_title(f'{name} sales plot')
    dset['sales'].plot()




In [0]:
list(set(groc_data1.family))

In [0]:
plt.rc('font',size=10)
grid = gridspec.GridSpec(3,2)
plt.figure(figsize=(19,15))
plt.subplots_adjust(wspace=0.4,hspace=0.3)
      
for idx, name in enumerate(list(set(groc_data1.family))):
    ax = plt.subplot(grid[idx])
    dset = groc_data1[groc_data1.family == name]
    dset['sales'] = dset['sales'] + 0.01

    result = seasonal_decompose(dset['sales'], model='multiplicative', period=365)
    ax.set_title(f'{name} Seasonal plot')
    result.seasonal.plot()



In [0]:
plt.rc('font',size=10)
grid = gridspec.GridSpec(3,2)
plt.figure(figsize=(19,15))
plt.subplots_adjust(wspace=0.4,hspace=0.3)
      
for idx, name in enumerate(list(set(groc_data1.family))):
    ax = plt.subplot(grid[idx])
    dset = groc_data1[groc_data1.family == name]
    dset['sales'] = dset['sales'] + 0.01

    result = seasonal_decompose(dset['sales'], model='multiplicative', period=365)
    ax.set_title(f'{name} Trend plot')
    result.trend.plot()
    


In [0]:
plt.rc('font',size=10)
grid = gridspec.GridSpec(3,2)
plt.figure(figsize=(19,15))
plt.subplots_adjust(wspace=0.4,hspace=0.3)
      
for idx, name in enumerate(list(set(groc_data1.family))):
    ax = plt.subplot(grid[idx])
    dset = groc_data1[groc_data1.family == name]
    dset['sales'] = dset['sales'] + 0.01
    result = seasonal_decompose(dset['sales'], model='multiplicative', period=365)
    ax.set_title(f'{name} Decomposition plot')
    result.plot();

Test/Train split

In [0]:
########################
### Test train split ###
########################

groc_data1['flag_prom'] = 1
groc_data1.loc[groc_data1.onpromotion == 0,'flag_prom'] = 0
groc_data1['y'] = groc_data1.sales
dset = groc_data1[groc_data1.family == list(set(groc_data1.family))[0]]

In [0]:
######################################
### Align data with Prophet format ###
######################################

time_data_ph = groc_data1[['date','sales','dcoilwtico','National','Regional','Local','flag_prom','Is_month_end','onpromotion','family',
                          'Is_year_start','Is_month_end','Dayofweek','Is_month_start','flag_prom']]

time_data_ph.columns = ['ds','y','dcoilwtico','National','Regional','Local','flag_prom','Is_month_end','onpromotion','family',
                       'Is_year_start','Is_month_end','Dayofweek','Is_month_start','flag_prom']
time_data_ph = time_data_ph.sort_values('ds')
time_data_ph['y'] = time_data_ph['y'].astype(int)
time_data_ph['ds'] = pd.to_datetime(time_data_ph['ds'], format='%Y-%m-%d')
time_data_ph = time_data_ph.reset_index(drop=True)

for name in ['National','Regional','Local','flag_prom','Is_month_end','Is_year_start','Is_month_end','Is_month_start']:
    time_data_ph[name] = time_data_ph[name].astype(int)

    
for idx, name in enumerate(list(set(groc_data1.family))):
    dset = time_data_ph[time_data_ph.family == name]
    y_train, y_test = temporal_train_test_split(dset['y'], test_size = 421)
    train_df = dset.iloc[:y_train.shape[0],]
    test_df = dset.iloc[y_train.shape[0]:,]
    print(train_df.shape,test_df.shape)
    fh = np.arange(1, len(y_test) + 1)   # forecasting horizon
    print(y_train.shape[0], y_test.shape[0])
    
    f = plt.figure(figsize=(19, 15))
    plot_series(y_train, y_test, labels=["y_train", "y_test"], title = f'{name} Train-test plot');
    
    plot_series(y_test, title = f'{name} Test plot');

In [0]:
#########################
### Plots of the data ###
#########################

#f = plt.figure(figsize=(19, 15))
#plot_series(y_train, y_test, labels=["y_train", "y_test"]);

In [0]:
#f = plt.figure(figsize=(19, 15));
#plot_series(y_test);

Prophet models

In [0]:
hdate = holidays_events.date.dt.date
a = list(set(train.month_date.dt.date))
a.sort()

dates = [x.replace(day = 15) for x in a]

holiday = pd.DataFrame({
  'holiday': 'holiday',
  'ds': pd.to_datetime(list(set(holi_list))),
  'lower_window': -1,
  'upper_window': 1,
})
pay_day = pd.DataFrame({
  'holiday': 'pay_day',
  'ds': pd.to_datetime(dates),
  'lower_window': -1,
  'upper_window': 1,
})
month_start = pd.DataFrame({
  'holiday': 'month_start',
  'ds': pd.to_datetime(a),
  'lower_window': -1,
  'upper_window': 1,
})
holidays = pd.concat((holiday, pay_day, month_start))
holidays = holidays.drop(['holiday'],axis = 1)
print(holidays.shape)
holidays = holidays.drop_duplicates()
print(holidays.shape)

for jan_date in ['2016-01-01','2017-01-01']:
    holidays.loc[holidays.ds == pd.to_datetime(jan_date),'lower_window'] = -4
    holidays.loc[holidays.ds == pd.to_datetime(jan_date),'upper_window'] = 4
holidays['holiday'] = 'Holiday'

In [0]:
### Growth terms ###

train_df['cap'] = max(int(max(train_df.y) * 1.1),int(max(test_df.y) * 1.1))
train_df['floor'] = 0
test_df['cap'] = max(int(max(train_df.y) * 1.1),int(max(test_df.y) * 1.1))
test_df['floor'] = 0

### Baseline model ###

m = Prophet()
m.fit(train_df)

### Strict model ###

m_strict = Prophet(changepoint_prior_scale=0.01,
                   weekly_seasonality=True, 
                   daily_seasonality=False, 
                   yearly_seasonality=True,
                   seasonality_mode = 'multiplicative')
m_strict.fit(train_df)

### Flexible model ###

m_flex = Prophet(changepoint_prior_scale=0.5)
m_flex.fit(train_df)

### Holiday model ###

m_hol = Prophet(holidays=holidays)
m_hol.add_regressor('dcoilwtico')
#m_hol.add_regressor('Is_month_end')
#m_hol.add_regressor('flag_prom')
m_hol.add_regressor('onpromotion')
m_hol.fit(train_df)

### Regressors ###

m_reg = Prophet(changepoint_prior_scale=0.01)
m_reg.add_regressor('dcoilwtico')
m_reg.add_regressor('National')
m_reg.add_regressor('Regional')
m_reg.add_regressor('Local')
m_reg.add_regressor('onpromotion')
m_reg.fit(train_df)

### Holiday-Regressors-Seasonality ###

m_hrs = Prophet(weekly_seasonality=True, 
                daily_seasonality=False, 
                yearly_seasonality=True,
                seasonality_mode = 'multiplicative',
                changepoint_prior_scale = 0.1, 
                seasonality_prior_scale = 10.0,
                holidays=holidays)
m_hrs.add_regressor('dcoilwtico')
#m_hrs.add_regressor('Is_month_end')
#m_hrs.add_regressor('flag_prom')
m_hrs.add_regressor('onpromotion')
m_hrs.fit(train_df)

### Holiday-Regressors-Seasonality ###

m_comp = Prophet(growth = 'linear',
                seasonality_mode = 'additive',      # seasonality_mode = 'multiplicative',
                changepoint_prior_scale = 0.1, 
                seasonality_prior_scale = 10.0,
                holidays_prior_scale = 20.0,
                weekly_seasonality = False, 
                daily_seasonality = False, 
                yearly_seasonality = False,
                holidays=holidays,
                ).add_seasonality(
                    name = 'monthly',
                    period = 30.5,
                    fourier_order = 55
                #).add_seasonality(
                    #name = 'daily',
                    #period = 1,
                    #fourier_order = 3,
                    #prior_scale = 30
                ).add_seasonality(
                    name = 'weekly',
                    period = 7,
                    fourier_order = 10,
                    prior_scale = 40
                ).add_seasonality(
                    name = 'yearly',
                    period = 365.25,
                    fourier_order = 20
                ).add_seasonality(
                    name = 'quarterly',
                    period = 365.25 / 4,
                    fourier_order = 5,
                    prior_scale = 15
                ).add_seasonality(
                    name = 'bi-monthly',
                    period = 365.25 / 6,
                    fourier_order = 5,
                    prior_scale = 15
                )

m_comp.add_regressor('dcoilwtico')
#m_hrs.add_regressor('Is_month_end')
#m_hrs.add_regressor('flag_prom')
m_comp.add_regressor('onpromotion')
m_comp.fit(train_df)


In [0]:
import logging
logger = logging.getLogger('cmdstanpy')
logger.addHandler(logging.NullHandler())
logger.propagate = False
logger.setLevel(logging.CRITICAL)

Baseline

In [0]:
df_cv, df_p, forecast, prop_test = cv_exam(m, test_df)

In [0]:
fig = m.plot_components(forecast)

In [0]:
mape_ph = mean_absolute_percentage_error(prop_test['y'], prop_test['yhat'], symmetric=True)
print(f"The Smape loss value is {mape_ph:.6f}")
# 0.124289
plot_series(prop_test['yhat'], prop_test['y'], labels=["y_pred","y_test"])

In [0]:
fig = plot_cross_validation_metric(df_cv, metric='smape')

In [0]:
f = plt.figure(figsize=(19, 15))
ax = sns.lineplot(x="ds", y="residual", markers=True, data=df_cv)
ax.set(xlabel='Dates', ylabel='Residuals')
plt.show()

Strict model

In [0]:
########################
### Cross validation ###
########################

df_cv_strict, df_p_strict, forecast_strict, prop_test_strict = cv_exam(m_strict, test_df)
fig = plot_cross_validation_metric(df_cv_strict, metric='smape')

In [0]:
fig = m_strict.plot(forecast_strict)
a = add_changepoints_to_plot(fig.gca(), m_strict, forecast_strict)

In [0]:
mape_ph_strict = mean_absolute_percentage_error(prop_test_strict['y'], prop_test_strict['yhat'], symmetric=True)
print(f"The Smape loss value is {mape_ph_strict:.6f}")
# 0.122349
plot_series(prop_test_strict['yhat'], y_test, labels=["y_pred","y_test"])

In [0]:
fig = m_strict.plot_components(forecast_strict)

In [0]:
f = plt.figure(figsize=(19, 15))
ax = sns.lineplot(x="ds", y="residual", markers=True, data=df_cv_strict)
ax.set(xlabel='Dates', ylabel='Residuals')
plt.show()

Flexible model

In [0]:
df_cv_flex, df_p_flex, forecast_flex, prop_test_flex = cv_exam(m_flex, test_df)
fig = plot_cross_validation_metric(df_cv_flex, metric='smape')

In [0]:
fig = m_flex.plot(forecast_flex)
a = add_changepoints_to_plot(fig.gca(), m_flex, forecast_flex)

In [0]:
mape_ph_flex = mean_absolute_percentage_error(prop_test_flex['y'], prop_test_flex['yhat'], symmetric=True)
print(f"The Smape loss value is {mape_ph_flex:.6f}")
# 0.123983
plot_series(prop_test_flex['yhat'], y_test, labels=["y_pred", "y_test"])

In [0]:
f = plt.figure(figsize=(19, 15))
ax = sns.lineplot(x="ds", y="residual", markers=True, data=df_cv_flex)
ax.set(xlabel='Dates', ylabel='Residuals')
plt.show()

Prophet with regressors

In [0]:
df_cv_reg, df_p_reg, forecast_reg, prop_test_reg = cv_exam(m_reg, test_df)

fig = plot_cross_validation_metric(df_cv_flex, metric='smape')

In [0]:
fig = m_reg.plot(forecast_reg)
a = add_changepoints_to_plot(fig.gca(), m_reg, forecast_reg)

In [0]:
mape_ph_reg = mean_absolute_percentage_error(prop_test_reg['y'], prop_test_reg['yhat'], symmetric=True)
print(f"The Smape loss value is {mape_ph_reg:.6f}")
# 0.124737
print(f"The RMSE value is {mean_squared_error(prop_test_reg['y'], prop_test_reg['yhat'], squared=False)}")
# 1344.3264089856723
plot_series(prop_test_reg['yhat'], y_test, labels=["y_pred", "y_test"])

In [0]:
f = plt.figure(figsize=(19, 15))
ax = sns.lineplot(x="ds", y="residual", markers=True, data=df_cv_reg)
ax.set(xlabel='Dates', ylabel='Residuals')
plt.show()

Prophet with holidays

In [0]:
df_cv_hol, df_p_hol, forecast_hol, prop_test_hol = cv_exam(m_hol, test_df)

fig = plot_cross_validation_metric(df_cv_hol, metric='smape')

In [0]:
fig = m_reg.plot(forecast_hol)
a = add_changepoints_to_plot(fig.gca(), m_hol, forecast_hol)

In [0]:
mape_ph_hol = mean_absolute_percentage_error(prop_test_hol['y'], prop_test_hol['yhat'], symmetric=True)
print(f"The Smape loss value is {mape_ph_hol:.6f}")
# 0.126283
print(f"The RMSE value is {mean_squared_error(prop_test_hol['y'], prop_test_hol['yhat'], squared=False)}")
# 1344.3264089856723
plot_series(prop_test_hol['yhat'], y_test, labels=["y_pred", "y_test"])

In [0]:
f = plt.figure(figsize=(19, 15))
ax = sns.lineplot(x="ds", y="residual", markers=True, data=df_cv_hol)
ax.set(xlabel='Dates', ylabel='Residuals')
plt.show()

Holiday-Regressors-Seasonality

In [0]:
df_cv_hrs, df_p_hrs, forecast_hrs, prop_test_hrs = cv_exam(m_hrs, test_df)

fig = plot_cross_validation_metric(df_cv_hrs, metric='smape')

In [0]:
fig = m_reg.plot(forecast_hrs)
a = add_changepoints_to_plot(fig.gca(), m_hrs, forecast_hrs)

In [0]:
mape_ph_hrs = mean_absolute_percentage_error(prop_test_hrs['y'], prop_test_hrs['yhat'], symmetric=True)
print(f"The Smape loss value is {mape_ph_hrs:.6f}")
# 0.126360
print(f"The RMSE value is {mean_squared_error(prop_test_hrs['y'], prop_test_hrs['yhat'], squared=False)}")
# 0.110473
plot_series(prop_test_hrs['yhat'], y_test, labels=["y_pred", "y_test"])

In [0]:
f = plt.figure(figsize=(19, 15))
ax = sns.lineplot(x="ds", y="residual", markers=True, data=df_cv_hrs)
ax.set(xlabel='Dates', ylabel='Residuals')
plt.show()

Constructed model

In [0]:
df_cv_comp, df_p_comp, forecast_comp, prop_test_comp = cv_exam(m_comp, test_df)

fig = plot_cross_validation_metric(df_cv_comp, metric='smape')

In [0]:
fig = m_reg.plot(forecast_comp)
a = add_changepoints_to_plot(fig.gca(), m_comp, forecast_comp)

In [0]:
mape_ph_comp = mean_absolute_percentage_error(prop_test_comp['y'], prop_test_comp['yhat'], symmetric=True)
print(f"The Smape loss value is {mape_ph_comp:.6f}")
# 0.125903
print(f"The RMSE value is {mean_squared_error(prop_test_comp['y'], prop_test_comp['yhat'], squared=False)}")
# 1210.0171287867122
plot_series(prop_test_comp['yhat'], y_test, labels=["y_pred", "y_test"])

In [0]:
f = plt.figure(figsize=(19, 15))
ax = sns.lineplot(x="ds", y="residual", markers=True, data=df_cv_comp)
ax.set(xlabel='Dates', ylabel='Residuals')
plt.show()

Compare models

In [0]:
df_p['Model'] = 'Baseline'
df_p_strict['Model'] = 'Strict'
df_p_flex['Model'] = 'Flexible'
df_p_reg['Model'] = 'Regressor'
df_p_hol['Model'] = 'Holiday'
df_p_hrs['Model'] = 'HRS'
df_p_comp['Model'] = 'Constructed'
df_comp = pd.concat([df_p,df_p_strict], axis = 0).reset_index(drop=True)
df_comp = pd.concat([df_comp,df_p_flex], axis = 0).reset_index(drop=True)
df_comp = pd.concat([df_comp,df_p_reg], axis = 0).reset_index(drop=True)
df_comp = pd.concat([df_comp,df_p_hol], axis = 0).reset_index(drop=True)
df_comp = pd.concat([df_comp,df_p_hrs], axis = 0).reset_index(drop=True)
df_comp = pd.concat([df_comp,df_p_comp], axis = 0).reset_index(drop=True)
df_comp['days'] = df_comp['horizon'].astype('timedelta64[D]')
df_comp['days'] = df_comp['days'].astype(int)

f = plt.figure(figsize=(19, 15))
ax = sns.lineplot(x="days", y="smape", hue="Model", markers=True, data=df_comp)
ax.set(xlabel='Date', ylabel='SMAPE')
plt.show()

In [0]:
perf_vals(df_comp,'smape')

In [0]:
perf_vals(df_comp,'rmse')

In [0]:
### Select best model rmse/smape ###




In [0]:
### Check out baseline measures ###

baseline_model_p = performance_metrics(df_cv_hol, rolling_window=1)
baseline_model_p.head()

Hyperparameter tuning on final model

In [0]:
holidays_df = pd.DataFrame(holidays[['ds','holiday']])
holidays_df.head()

In [0]:
import itertools

### Set up parameter grid ###

param_grid = {  
    'changepoint_prior_scale': [0.001, 0.05, 0.08, 0.5],
    'seasonality_prior_scale': [0.01, 1, 5, 10, 12],
    'seasonality_mode': ['multiplicative','additive'],
    'weekly_seasonality': [True],
    'yearly_seasonality': [True],
    'daily_seasonality': [False],
    'holidays': [holidays_df]
}

### Generate all combinations of parameters ###

all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]

### Create a list to store MAPE values for each combination ###
mapes = [] 

### Use cross validation to evaluate all parameters ###

for params in all_params:
    ### Fit a model using one parameter combination ###
    m = Prophet(**params)
    m.add_regressor('dcoilwtico')
    m.add_regressor('onpromotion')
    m.fit(time_data_ph)
    
    ### Cross-validation ###
    df_cv = cross_validation(m, initial='730 days', period='15 days', horizon = '30 days', parallel="processes")
    
    ### Model performance ###
    df_p = performance_metrics(df_cv, rolling_window=1)
    
    ### Save model performance metrics ###
    mapes.append(df_p['smape'].values[0])
    
### Tuning results
tuning_results = pd.DataFrame(all_params)
tuning_results['mape'] = mapes
# Find the best parameters
best_params = all_params[np.argmin(mapes)]
print(best_params)

In [0]:
### Fit the model using the best parameters ###

auto_model = Prophet(changepoint_prior_scale = best_params['changepoint_prior_scale'], 
                     seasonality_prior_scale = best_params['seasonality_prior_scale'], 
                     seasonality_mode = best_params['seasonality_mode'],
                     holidays = holidays)
auto_model.add_regressor('dcoilwtico')
auto_model.add_regressor('onpromotion')

### Fit the model on the training dataset ###

auto_model.fit(time_data_ph)

### Cross validation ###
auto_model_cv, auto_model_p, forecast_auto, prop_test_auto = cv_exam(auto_model, test_df)

### Model performance metrics ###
auto_model_pm = performance_metrics(auto_model_cv, rolling_window=1)
auto_model_pm

In [0]:
auto_model_p['Model'] = 'Hyperparameter'

df_comp = pd.concat([df_comp,auto_model_p], axis = 0).reset_index(drop=True)
df_comp['days'] = df_comp['horizon'].astype('timedelta64[D]')
df_comp['days'] = df_comp['days'].astype(int)

f = plt.figure(figsize=(19, 15))
ax = sns.lineplot(x="days", y="smape", hue="Model", markers=True, data=df_comp)
ax.set(xlabel='Date', ylabel='SMAPE')
plt.show()

In [0]:
fig = m_reg.plot(forecast_auto)
a = add_changepoints_to_plot(fig.gca(), auto_model, forecast_auto)

In [0]:
mape_ph_auto = mean_absolute_percentage_error(prop_test_auto['y'], prop_test_auto['yhat'], symmetric=True)
print(f"The Smape loss value is {mape_ph_auto:.6f}")
# 0.111345
print(f"The RMSE value is {mean_squared_error(prop_test_auto['y'], prop_test_auto['yhat'], squared=False)}")
# 1186.9104787999713
plot_series(prop_test_auto['yhat'], y_test, labels=["y_pred", "y_test"])

In [0]:
f = plt.figure(figsize=(19, 15))
ax = sns.lineplot(x="ds", y="residual", markers=True, data=auto_model_cv)
ax.set(xlabel='Dates', ylabel='Residuals')
plt.show()

Comparison

In [0]:
auto_model_p['Model'] = 'Hyperparameter'

df_comp = pd.concat([df_comp,auto_model_p], axis = 0).reset_index(drop=True)
df_comp['days'] = df_comp['horizon'].astype('timedelta64[D]')
df_comp['days'] = df_comp['days'].astype(int)

f = plt.figure(figsize=(19, 15))
ax = sns.lineplot(x="days", y="smape", hue="Model", markers=True, data=df_comp)
ax.set(xlabel='Date', ylabel='SMAPE')
plt.show()

In [0]:
perf_vals(df_comp, meas = 'smape')

In [0]:
perf_vals(df_comp, meas = 'rmse')